In [51]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
# confusion matrix
from sklearn.metrics import confusion_matrix

In [48]:
# df = pd.read_csv('../../Data/heart_2022_with_nans.csv')

# sample 500 rows from df
# final = df.dropna()

# final['HadHeartDisease'] = final.apply(lambda row: 'Yes' if 'Yes' in [row['HadHeartAttack'], 
#                                                                           row['HadStroke'], 
#                                                                           row['HadAngina']] else 'No', 
#                                                                           axis=1)

# # only keep columns that contain yes or no values
# final = final[['PhysicalHealthDays', 'MentalHealthDays', 
#               'PhysicalActivities','SleepHours', 
#               'HadHeartAttack', 'HadAngina', 
#               'HadStroke', 'HadAsthma',
#               'HadSkinCancer', 'HadCOPD', 
#               'HadDepressiveDisorder', 'HadKidneyDisease',
#               'HadArthritis','HadHeartDisease']]
# final

<ipython-input-48-829ef29e1041>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['HadHeartDisease'] = final.apply(lambda row: 'Yes' if 'Yes' in [row['HadHeartAttack'],


,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadHeartDisease
342,4.0,0.0,Yes,9.0,No,No,No,No,No,No,No,No,Yes,No
343,0.0,0.0,Yes,6.0,No,No,No,No,No,No,No,No,Yes,No
345,0.0,0.0,No,8.0,No,No,No,No,No,No,No,No,Yes,No
346,5.0,0.0,Yes,9.0,No,No,No,No,Yes,No,Yes,No,Yes,No
347,3.0,15.0,Yes,5.0,No,No,No,No,No,No,No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445117,0.0,0.0,Yes,6.0,No,No,No,No,No,No,No,No,No,No
445123,0.0,7.0,Yes,7.0,No,No,No,No,No,No,Yes,No,No,No
445124,0.0,15.0,Yes,7.0,No,No,Yes,No,No,No,No,No,Yes,Yes
445128,2.0,2.0,Yes,7.0,No,No,No,No,No,No,No,No,No,No


In [55]:
# prepare data for logistic regression
def logi_reg(df,target, target_names, thresh_n):
    """You know what it does 

        Input: 
            df: pandas dataframe
            target: target column name
            target_names: the names of the target classes eg."Yes" and "No"

        Output:
            None 
    """

    # turn all anything containing Yes or No columns to binary
    # simplify all entries to Yes or No
    df = df.replace({'Yes': 1, 'No': 0})
    df_train, df_test = train_test_split(df, 
                                        test_size = 0.25, 
                                        random_state = 69,
                                        stratify = df[target])

    X_train = df_train.drop(columns = [target])
    y_train = df_train[target]
    X_test = df_test.drop(columns = [target])
    y_test = df_test[target]

    # print the size of the training and test set
    print (f"The threshold for removing NAs is {thresh_n}")
    print (f"Size of training set : {df_train.shape[0]} rows , {df_train.shape[1]} columns")
    print (f"Size of testing set : {df_test.shape[0]} rows , {df_test.shape[1]} columns")

    # fit logistic regression model with elastic net regularization
    log_reg = LogisticRegression(penalty = 'elasticnet',
                                solver = 'saga',
                                l1_ratio = 0.5,
                                max_iter = 1000)
    
    log_reg.fit(X_train, y_train)

    # predict on test set
    y_test_pred = log_reg.predict(X_test)
    y_pred_proba = log_reg.predict_proba(X_test)[:,1]

    # print metrics
    print('\n===============================\n'+
      'Classification report on test data' +
      '\n===============================\n')
    print(classification_report(y_test, y_test_pred, target_names=target_names))

    print('\n===============================\n'+
        'Confusion matrix on test data' +
        '\n===============================\n')
    print(confusion_matrix(y_test, y_test_pred))



In [56]:
logi_reg(final, 'HadHeartDisease', ['No', 'Yes'], 0.5)

The threshold for removing NAs is 0.5
Size of training set : 184516 rows , 14 columns
Size of training set : 61506 rows , 14 columns

Classification report on test data

              precision    recall  f1-score   support

          No       1.00      1.00      1.00     54451
         Yes       1.00      1.00      1.00      7055

    accuracy                           1.00     61506
   macro avg       1.00      1.00      1.00     61506
weighted avg       1.00      1.00      1.00     61506


Confusion matrix on test data

[[54451     0]
 [    0  7055]]
